In [1]:
import os
import pandas as pd
from tqdm import tqdm
import requests
import json
from openai import OpenAI
import configparser
import argparse

In [ ]:
def query_model_openrouter(model_name, config, task_file, retry = 3):
    client = OpenAI(api_key=config["OPENROUTER"]["key"], base_url=config["OPENROUTER"]["url"])
    df_task = pd.read_csv(task_file, encoding='utf-8')

    task_name = task_file.split('/')[-1].split('.')[0]
    questions = list(df_task["instruction"] + "\n\n" + df_task["question"])

    results = []
    for question in tqdm(questions, desc=task_name):
        success = False
        trial = 0
        while(not success and trial < retry):
            try:
                response = client.chat.completions.create(
                    model=model_name,
                    temperature=float(config["PARAMS"]["temperature"]),
                    frequency_penalty=float(config["PARAMS"]["frequency_penalty"]), 
                    presence_penalty=float(config["PARAMS"]["presence_penalty"]),
                    top_p=float(config["PARAMS"]["top_p"]), 
                    max_tokens=int(config["PARAMS"]["max_tokens"]), 
                    messages=[{"role": "user", "content": question}],
                )
                llm_result = response.choices[0].message.content

                # response = requests.post(
                #     url=config["OPENROUTER"]["url"] + "/chat/completions",
                #     headers={
                #         "Authorization": "Bearer " + config["OPENROUTER"]["key"],
                #     },
                #     data=json.dumps({
                #         "model": model_name, 
                #         "messages": [
                #         {
                #             "role": "user",
                #             "content": question
                #         }
                #         ],
                #         "provider":{
                #             "order": ["Together", ], 
                #         },
                #         "temperature":float(config["PARAMS"]["temperature"]),
                #         "frequency_penalty":float(config["PARAMS"]["frequency_penalty"]),
                #         "presence_penalty":float(config["PARAMS"]["presence_penalty"]),
                #         "top_p":float(config["PARAMS"]["top_p"]),
                #         "max_tokens":int(config["PARAMS"]["max_tokens"]),
                #     }),
                #     timeout=180
                # )
                # response = response.json()
                # llm_result = response["choices"][0]["message"]["content"]

                results.append((question, llm_result))
                success = True
                trial += 1
            except Exception as e:
                success = False
                trial += 1
                print(f"Trial:{trial}\nError log:\n{e}")
                continue
        if not success:
            results.append((question, "API ERROR"))

    df_results = pd.DataFrame(results, columns=["prompt", "response"])
    df_merge = pd.concat([df_task, df_results], axis=1)

    return df_merge